<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/SingleSubjectSVM_Norm_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration of Single Subject SVM
- Go to 'Runtime' in Colab browser bar, select 'Change Runtime Type', select 'High-RAM' from 'Runtime Shape'. 
- This notebook runs exploration of different normalization strategies for brain data and performs cross-validation. You can visualize and analyze data in place or you can save the data and do it in a separate notebook. This will demonstrate how to do it in place. You will need to pull in single_subject.py and 
> Normalization Strategies
  1. Choose run(s) to train on
  2. Choose run(s) to test on
  3. Choose from the following for normalization ['psc','zscore','nonorm'].
    * This is a list of of normalizations to apply to the data. It will iterate through the list.
  4. Choose params for C, kernel, gamma. If not specified, default will be used.
  5. Choose list of subjects to save. I saved one adult and one child for visualizations
  6. Choose output file name.
  7. The cell for normalization will iterate over all subjects unless, you choose a subset.
  8. Output will be a pandas dataframe with accuracy and F1 scores of the subjects.
  9. The subject models variables are stored in a dictionary for further analysis and visualiztions.
    * Variables Stored 
      1. clf
      2. X-train
      3. y-train (even though all runs have same labels, we want to store this separately in case we train on two runs where labels will be concatenated).
      4. y_labels (since we test on single runs and all runs have the same data, we just need on set of labels)
      5. scaled data of each run so we can look at decision function boundaries
      6. Since the data has been transformed and stripped, we can save this locally. You can either choose to analyze immediatly or save data for analysis later.
10. Create summary statistics for all subjects. I also chose to look at differences between adolescents and young adults.
11. Create boxplots.
12. Create heatmaps of brains based on normalization.
13. Look at results of decision function scores overlaying on increase/decrease trials within a run.

> Cross-validation
  * Once we have the best normalization for our data, run the cross validation
    1. Choose the run(s) to perform cross validation on.
    2. Choose runs to get accuracy measures on from the cv on the subjects(?)
    3. Choose params you want to do cross-validation on.
    4. Choose output destination. If further analysis is needed.
      * Output is a dictionary containing the model for each subject.
    5. Loop through subjects to get results of cross validation.
      * Data is stored in a data frame.
    6. Create boxplot of results.






### Mount Google Drive and clone repository
- open to source directory

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/single_subject
!ls


Cloning into 'teambrainiac'...
remote: Enumerating objects: 1453, done.
remote: Counting objects: 100% (1453/1453), done.
remote: Compressing objects: 100% (1151/1151), done.
remote: Total 1453 (delta 944), reused 557 (delta 285), pack-reused 0
Receiving objects: 100% (1453/1453), 84.71 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (944/944), done.
/content/teambrainiac/source
access_data.py			    __init__.py
AccuracyMeasures.ipynb		    models
brain_viz_single_subj.py	    process.py
BuildSingleSubjectSVM_Models.ipynb  single_subject.py
data				    SingleSubjectSVM.ipynb
DataExploration_SingleSubj.ipynb    streamlit
DataExplorationVisuals.ipynb	    SubjectVisualization_Models_ZNORM.ipynb
DL				    TestMask.ipynb
group_svm			    utils.py
helper				    VisualizationPlayground.ipynb
images


### Load path_config.py 
- we are already in source so we can just load this file without changing directory

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


### Import libraries


In [ ]:


# Import libraries
!pip install boto3 nilearn nibabel #for saving data and image visualizations
import pickle
#sklearn packages needed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, auc, recall_score, precision_score,roc_curve,f1_score
#important utility functions for loading,masking,saving data
from access_data import *
#single subject model building functions
from single_subject import *
#for visualization(s)
from brain_viz_single_subj import *

#normal python packages we use
import numpy as np
import pandas as pd
from scipy import signal
from nilearn.signal import clean

     |████████████████████████████████| 132 kB 4.2 MB/s 
     |████████████████████████████████| 9.6 MB 12.8 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 8.7 MB 48.0 MB/s 
     |████████████████████████████████| 138 kB 63.1 MB/s 
     |████████████████████████████████| 38.1 MB 359 kB/s 
     |████████████████████████████████| 127 kB 46.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 r

### Get paths to subject data

In [ ]:

## load and open the pickle file that contains paths to all data.
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

##get mask_dictionary


Run Normalization

In [ ]:

###data exploration for different normalization strategies
##what runs do you want to normalize on
runs_train=['run_02'] #runs we want to train on
runs_test=['run_03','run_04'] #runs we want to test on
runs_list=[1,2,3,4]
##for fMRI, we always want to detrend the data
norm_list = ['psc','zscore','none'] #list of normalization strategies you want to test
save_subject_data =  ['10047_09030','30017_09567'] #specify subjects you want to save for normalization visualizations
svc_kernel='rbf' #specify kernel 
svc_c = 1 #specify c parameter
svc_gamma='scale' #specify gamma parameter
mask_dict = get_mask_data(data_path_dict,0) #get mask_dict
mask = make_mask(mask_dict['mask']) #create mask
#get subject information
subjs_id, subjs_paths = get_subj_information(data_path_dict)
#get mask labels to only retrieve time series we care about
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
###initialize variables to hold results
model_dict = {}
results = []
df_columns = ['subject_id','norm_type']
#loop over norm_type to get initialize model_dict norm_type dictionary
for norm_type in norm_list:
  model_dict[norm_type] = {}
#for loop to loop over subjects
for idx in range(len(subjs_id)):
  subj_id = subjs_id[idx] #get subj_id
  subj_path = subjs_paths[idx] #get subj_path
  subj_data = access_load_data(subj_path,True) #get subj_data
  masked_data = mask_subject_data(subj_data,mask,mask_labels_indices) #mask subject data
  #loop over normalizations to do
  for norm_type in norm_list:
    norm_results = [subj_id,norm_type] #store subject id and normalization strategy in results list
    scaled_data = scale_data_single_subj(masked_data,runs_list,norm=norm_type) #scale the data for runs listed in runs_lit
    clf,X_train,y_train = run_single_subject_svm(scaled_data,runs_train,binary_labels,svc_kernel,svc_c) #run the model
    if subj_id in save_subject_data:    #if subject in list of subjects to save
      model_dict[norm_type][subj_id] = {} #initialize empty dictionary for the subject data
      #store subject variables
      model_dict[norm_type][subj_id]['model'] = clf
      model_dict[norm_type][subj_id]['X_train'] = X_train
      model_dict[norm_type][subj_id]['y_train'] = y_train
      model_dict[norm_type][subj_id]['y_test'] = binary_labels
      model_dict[norm_type][subj_id]['data'] = scaled_data
    sub_scores,cols = get_accuracy_scores(clf,scaled_data,X_train,y_train,runs_test,binary_labels) #get accuracy scores for analysis
    norm_results.extend(sub_scores) #append to list
    results.append(norm_results) #append to results
df_columns.extend(cols) #extend data frame columns
results_df = pd.DataFrame(results,columns=df_columns) #create df
#save results for analysis
save_data_path = f'/content/drive/My Drive/data/dataexploration/{svc_kernel}_exploration/accuracy_results.csv'
results_df.to_csv(save_data_path)
#save models for visualization
save_models_path = f'/content/drive/My Drive/data/dataexploration/{svc_kernel}_exploration/norm_models.pkl'
filehandler = open(save_models_path,"wb")
pickle.dump(model_dict,filehandler)
filehandler.close()



100%|██████████| 4/4 [00:01<00:00,  3.52it/s]
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:83: RuntimeWarning: invalid value encountered in true_divide
  signals = (signals - mean_signal) / np.absolute(mean_signal)
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:87: UserWarning: psc standardization strategy is meaningless for features that have a mean of 0. These time series are set to 0.
  warnings.warn('psc standardization strategy is meaningless '


None
None
None


100%|██████████| 4/4 [00:01<00:00,  3.68it/s]


None
None
None


100%|██████████| 4/4 [00:01<00:00,  3.66it/s]
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:83: RuntimeWarning: invalid value encountered in true_divide
  signals = (signals - mean_signal) / np.absolute(mean_signal)
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:87: UserWarning: psc standardization strategy is meaningless for features that have a mean of 0. These time series are set to 0.
  warnings.warn('psc standardization strategy is meaningless '


...

None
None
None


100%|██████████| 4/4 [00:00<00:00,  4.02it/s]
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:83: RuntimeWarning: invalid value encountered in true_divide
  signals = (signals - mean_signal) / np.absolute(mean_signal)
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:87: UserWarning: psc standardization strategy is meaningless for features that have a mean of 0. These time series are set to 0.
  warnings.warn('psc standardization strategy is meaningless '


None
None
None


100%|██████████| 4/4 [00:00<00:00,  4.05it/s]
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:83: RuntimeWarning: invalid value encountered in true_divide
  signals = (signals - mean_signal) / np.absolute(mean_signal)
/usr/local/lib/python3.7/dist-packages/nilearn/signal.py:87: UserWarning: psc standardization strategy is meaningless for features that have a mean of 0. These time series are set to 0.
  warnings.warn('psc standardization strategy is meaningless '


None
None
None


In [ ]:
cv_dict = {} #initialize the results dictionary
destination_path = '/content/drive/My Drive/data/dataexploration/cross_validation_results/cv_results_short.pkl' #where to save data
runs_train=['run_01','run_02'] #runs we want to train on
runs_list=[1,2,3,4] #runs we want to do cv on
runs_test=['run_03','run_04']
cv_params = {'C':[1, 5, 10,1000],'gamma': ['auto','scale']} #params we want to test
subjs_id,subjs_paths = get_subj_information(data_path_dict) #get subject information
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict) #get labels
mask_dict = get_mask_data(data_path_dict,0) #get mask dictionary containing mask data
mask = make_mask(mask_dict['mask']) #mask we want to use in cv
norm_type = 'zscore' #which normalization we want
#iterate over subjects and perform cv single subject svm
for idx in range(len(subject_ids)):
  subj_id = subjs_id[idx] #get subject id
  subj_path = subjs_paths[idx] #get path to subject data
  subj_data = access_load_data(subj_path,True)  #load subject data
  masked_data = mask_subject_data(subj_data,mask,mask_labels_indices) #mask the data
  scaled_data = scale_data_single_subj(masked_data,runs_list,norm=norm_type) #normalize the data
  clf = run_single_subject_svm(scaled_data,runs_train,binary_labels,do_cv=True,params=cv_params) #run cross validation
  cv_dict[subj_id] = {} #initialize subject dictionary
  cv_dict[subj_id]['model'] = clf #save cv model for further analysis
  y_predicts = clf.predict(scaled_data['run_03'])
  cv_dict[subj_id]['run_03_acc'] = accuracy_score(binary_labels,y_predicts)
  y_predicts = clf.predict(scaled_data['run_04'])
  cv_dict[subj_id]['run_04_acc'] = accuracy_score(binary_labels,y_predicts)
#save data
filehandler = open(destination_path,"wb")
pickle.dump(cv_dict,filehandler)
filehandler.close() 

100%|██████████| 4/4 [00:00<00:00,  4.92it/s]


{'C': [1, 5, 10, 1000], 'gamma': ['auto', 'scale']}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...................C=1, gamma=auto;, score=0.794 total time=   4.0s
[CV 2/5] END ...................C=1, gamma=auto;, score=0.676 total time=   4.1s
[CV 3/5] END ...................C=1, gamma=auto;, score=0.765 total time=   4.1s
[CV 4/5] END ...................C=1, gamma=auto;, score=0.970 total time=   4.1s
[CV 5/5] END ...................C=1, gamma=auto;, score=0.697 total time=   4.0s
[CV 1/5] END ..................C=1, gamma=scale;, score=0.794 total time=   4.2s
[CV 2/5] END ..................C=1, gamma=scale;, score=0.676 total time=   4.2s
[CV 3/5] END ..................C=1, gamma=scale;, score=0.765 total time=   4.4s
[CV 4/5] END ..................C=1, gamma=scale;, score=0.970 total time=   4.3s
[CV 5/5] END ..................C=1, gamma=scale;, score=0.697 total time=   4.1s
[CV 1/5] END ...................C=5, gamma=auto;, score=0.794 total time=   4.

100%|██████████| 4/4 [00:01<00:00,  2.86it/s]


{'C': [1, 5, 10, 1000], 'gamma': ['auto', 'scale']}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...................C=1, gamma=auto;, score=0.647 total time=   3.8s
[CV 2/5] END ...................C=1, gamma=auto;, score=0.971 total time=   4.0s
[CV 3/5] END ...................C=1, gamma=auto;, score=0.588 total time=   4.0s
[CV 4/5] END ...................C=1, gamma=auto;, score=0.879 total time=   3.8s
[CV 5/5] END ...................C=1, gamma=auto;, score=0.879 total time=   3.8s
[CV 1/5] END ..................C=1, gamma=scale;, score=0.647 total time=   3.9s
[CV 2/5] END ..................C=1, gamma=scale;, score=0.971 total time=   3.9s
[CV 3/5] END ..................C=1, gamma=scale;, score=0.588 total time=   4.1s
[CV 4/5] END ..................C=1, gamma=scale;, score=0.879 total time=   3.8s
[CV 5/5] END ..................C=1, gamma=scale;, score=0.879 total time=   4.0s
[CV 1/5] END ...................C=5, gamma=auto;, score=0.647 total time=   3.

100%|██████████| 4/4 [00:00<00:00,  7.07it/s]


...

{'C': [1, 5, 10, 1000], 'gamma': ['auto', 'scale']}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...................C=1, gamma=auto;, score=0.853 total time=   4.6s
[CV 2/5] END ...................C=1, gamma=auto;, score=0.941 total time=   4.5s
[CV 3/5] END ...................C=1, gamma=auto;, score=0.794 total time=   4.4s
[CV 4/5] END ...................C=1, gamma=auto;, score=0.970 total time=   4.2s
[CV 5/5] END ...................C=1, gamma=auto;, score=0.970 total time=   4.4s
[CV 1/5] END ..................C=1, gamma=scale;, score=0.853 total time=   4.7s
[CV 2/5] END ..................C=1, gamma=scale;, score=0.941 total time=   4.6s
[CV 3/5] END ..................C=1, gamma=scale;, score=0.794 total time=   4.7s
[CV 4/5] END ..................C=1, gamma=scale;, score=0.970 total time=   4.3s
[CV 5/5] END ..................C=1, gamma=scale;, score=0.970 total time=   4.4s
[CV 1/5] END ...................C=5, gamma=auto;, score=0.824 total time=   4.

100%|██████████| 4/4 [00:01<00:00,  3.64it/s]


{'C': [1, 5, 10, 1000], 'gamma': ['auto', 'scale']}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...................C=1, gamma=auto;, score=0.647 total time=   3.9s
[CV 2/5] END ...................C=1, gamma=auto;, score=0.676 total time=   4.1s
[CV 3/5] END ...................C=1, gamma=auto;, score=0.559 total time=   4.0s
[CV 4/5] END ...................C=1, gamma=auto;, score=0.939 total time=   4.1s
[CV 5/5] END ...................C=1, gamma=auto;, score=1.000 total time=   4.1s
[CV 1/5] END ..................C=1, gamma=scale;, score=0.647 total time=   4.3s
[CV 2/5] END ..................C=1, gamma=scale;, score=0.676 total time=   4.3s
[CV 3/5] END ..................C=1, gamma=scale;, score=0.559 total time=   4.2s
[CV 4/5] END ..................C=1, gamma=scale;, score=0.939 total time=   4.3s
[CV 5/5] END ..................C=1, gamma=scale;, score=1.000 total time=   4.4s
[CV 1/5] END ...................C=5, gamma=auto;, score=0.647 total time=   4.

100%|██████████| 4/4 [00:01<00:00,  3.72it/s]


{'C': [1, 5, 10, 1000], 'gamma': ['auto', 'scale']}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...................C=1, gamma=auto;, score=0.500 total time=   4.1s
[CV 2/5] END ...................C=1, gamma=auto;, score=0.706 total time=   4.1s
[CV 3/5] END ...................C=1, gamma=auto;, score=0.676 total time=   4.1s
[CV 4/5] END ...................C=1, gamma=auto;, score=0.788 total time=   4.1s
[CV 5/5] END ...................C=1, gamma=auto;, score=0.545 total time=   4.0s
[CV 1/5] END ..................C=1, gamma=scale;, score=0.500 total time=   4.2s
[CV 2/5] END ..................C=1, gamma=scale;, score=0.706 total time=   4.3s
[CV 3/5] END ..................C=1, gamma=scale;, score=0.676 total time=   4.1s
[CV 4/5] END ..................C=1, gamma=scale;, score=0.818 total time=   4.2s
[CV 5/5] END ..................C=1, gamma=scale;, score=0.545 total time=   4.1s
[CV 1/5] END ...................C=5, gamma=auto;, score=0.588 total time=   4.

In [ ]:
scaled_data

{'run_02': array([[ 0.        ,  0.        ,  0.        , ..., -0.35537407,
         -0.28912827, -0.15646337],
        [ 0.        ,  0.        ,  0.        , ..., -0.3566215 ,
         -0.29077935, -0.16021122],
        [ 0.        ,  0.        ,  0.        , ...,  2.8391807 ,
          2.9680855 ,  1.7116985 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  2.7306352 ,
          2.807277  ,  1.3825058 ],
        [ 0.        ,  0.        ,  0.        , ...,  2.780192  ,
          2.8530087 ,  1.4065295 ],
        [ 0.        ,  0.        ,  0.        , ...,  1.2044214 ,
          0.02133828, -0.35965276]], dtype=float32),
 'run_03': array([[ 0.        ,  0.        ,  0.        , ...,  0.1804529 ,
          0.2650536 , -0.84340256],
        [ 0.        ,  0.        ,  0.        , ..., -1.8602828 ,
         -2.2273705 , -1.2860752 ],
        [ 0.        ,  0.        ,  0.        , ..., -1.5041631 ,
         -1.92538   , -1.2338098 ],
        ...,
        [ 0.       